In [1]:
import pandas as pd

In [2]:
df = pd.read_json("output/QA_polaris_md_llama3-chatqa:70b_2000_3.json")
df.head()

,id,context,golden_context,question,cot_answer,answer
0,data/md/polaris/getting-started.md_seed_task_0_0,[Getting Started on Polaris\n\nLogging Into Po...,Getting Started on Polaris\n\nLogging Into Pol...,How do I log into Polaris?,"Reasoning:\n\nTo log into Polaris, I will foll...","ssh <username>@polaris.alcf.anl.gov, then type..."
1,data/md/polaris/getting-started.md_seed_task_0_1,"[Currently, MIG configuration is only availabl...",Getting Started on Polaris\n\nLogging Into Pol...,What is the purpose of Lustre File Striping in...,Reasoning:\n\nTo find the purpose of Lustre Fi...,The purpose of Lustre File Striping in Polaris...
2,data/md/polaris/getting-started.md_seed_task_0_2,[Known Issues\n\nThis is a collection of known...,Getting Started on Polaris\n\nLogging Into Pol...,Where can I find examples of job submission sc...,To find examples of job submission scripts for...,Example Job Scripts page
3,data/md/polaris/system-updates.md_seed_task_2_0,"[3. Use ParaView\n\nAfter the previous step, y...",Datascience Anaconda Module Updates\n\nWe have...,What CUDA version is compatible with the new P...,"To answer this question, I will follow these s...",CUDA 12.2
4,data/md/polaris/system-updates.md_seed_task_2_1,[ParaView on Polaris\n\nThe recommended way of...,Datascience Anaconda Module Updates\n\nWe have...,Which Python libraries are available as part o...,"To answer this question, I will follow these s...","PyTorch 2.3.0, TensorFlow 2.16.1, and other po..."


In [ ]:
system = "This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
texts = []
for index, row in df.iterrows():
    docs = row['context']
    context = ""
    for doc in docs:
        context += "<DOCUMENT>" + str(doc) + "</DOCUMENT>\n"
    question = row['question']
    answer = row['answer']
    text = """System: {system}\nContext: {context}\nUser: {question}\nAssistant: {answer}
    """.format(system = system, context = context, question = question, answer = answer)

    texts.append(text)

new_df = pd.DataFrame(texts, columns = ['text'])
new_df.to_csv("./data/train/nvidia/nvidia_train.csv",index=False)
new_df

,text
0,System: This is a chat between a user and an a...
1,System: This is a chat between a user and an a...
2,System: This is a chat between a user and an a...
3,System: This is a chat between a user and an a...
4,System: This is a chat between a user and an a...
...,...
312,System: This is a chat between a user and an a...
313,System: This is a chat between a user and an a...
314,System: This is a chat between a user and an a...
315,System: This is a chat between a user and an a...


In [6]:
system = "This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the given context. The assistant should also indicate when the answer cannot be found in the context."
texts = []
for index, row in df.iterrows():
    context = row['golden_context']
    question = row['question']
    answer = row['answer']
    text = """### Human: {question}### Assistant: {answer}""".format(question = question, answer = answer)

    texts.append(text)

df_gc = pd.DataFrame(texts, columns = ['text'])
df_gc.to_csv("./data/train/QA_only/train_with_QA_only.csv")
df_gc

,text
0,### Human: How do I log into Polaris?### Assis...
1,### Human: What is the purpose of Lustre File ...
2,### Human: Where can I find examples of job su...
3,### Human: What CUDA version is compatible wit...
4,### Human: Which Python libraries are availabl...
...,...
312,### Human: How can I compile a simple code lik...
313,### Human: What are some examples of job scrip...
314,### Human: How do you determine the number of ...
315,### Human: Can you provide an example of how t...


In [9]:
# system = "This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the given context. The assistant should also indicate when the answer cannot be found in the context."
import json
with open("./data/train/QA_only/train_with_QA_only.jsonl", "w") as f:
    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        data = {"text": """### Human: {question}### Assistant: {answer}""".format(question = question, answer = answer)}

        f.write(json.dumps(data) + "\n")